In [794]:
# Import library
import pandas as pd
import numpy as np
from datetime import datetime

# Load datasets
df = pd.read_csv('fifa21_raw_data.csv', low_memory=False)
print(f"Initial Data: {df.shape}")
df.head()

Initial Data: (18979, 77)


,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,ID,Height,Weight,foot,BOV,BP,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,https://cdn.sofifa.com/players/158/023/21_60.png,Lionel Messi,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,RW ST CF,L. Messi,33,93,93,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n,158023,"5'7""",159lbs,Left,93,RW,0,"Jul 1, 2004",NaN,€67.5M,€560K,€138.4M,429,85,95,70,91,88,470,96,93,94,91,96,451,91,80,91,94,95,389,86,68,72,69,94,347,44,40,93,95,75,96,91,32,35,24,54,6,11,15,14,8,2231,466,4 ★,4★,Medium,Low,5 ★,85,92,91,95,38,65,\n372
1,https://cdn.sofifa.com/players/020/801/21_60.png,C. Ronaldo dos Santos Aveiro,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,ST LW,Cristiano Ronaldo,35,92,92,\n\n\n\nJuventus\n2018 ~ 2022\n\n,20801,"6'2""",183lbs,Right,92,ST,0,"Jul 10, 2018",NaN,€46M,€220K,€75.9M,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11,15,14,11,2221,464,4 ★,5★,High,Low,5 ★,89,93,81,89,35,77,\n344
2,https://cdn.sofifa.com/players/200/389/21_60.png,Jan Oblak,http://sofifa.com/player/200389/jan-oblak/210005/,Slovenia,GK,J. Oblak,27,91,93,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n,200389,"6'2""",192lbs,Right,91,GK,2,"Jul 16, 2014",NaN,€75M,€125K,€159.4M,95,13,11,15,43,13,109,12,13,14,40,30,307,43,60,67,88,49,268,59,78,41,78,12,140,34,19,11,65,11,68,57,27,12,18,437,87,92,78,90,90,1413,489,3 ★,1★,Medium,Medium,3 ★,87,92,78,90,52,90,\n86
3,https://cdn.sofifa.com/players/192/985/21_60.png,Kevin De Bruyne,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,CAM CM,K. De Bruyne,29,91,91,\n\n\n\nManchester City\n2015 ~ 2023\n\n,192985,"5'11""",154lbs,Right,91,CAM,0,"Aug 30, 2015",NaN,€87M,€370K,€161M,407,94,82,55,94,82,441,88,85,83,93,92,398,77,76,78,91,76,408,91,63,89,74,91,408,76,66,88,94,84,91,186,68,65,53,56,15,13,5,10,13,2304,485,5 ★,4★,High,High,4 ★,76,86,93,88,64,78,\n163
4,https://cdn.sofifa.com/players/190/871/21_60.png,Neymar da Silva Santos Jr.,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,LW CAM,Neymar Jr,28,91,91,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n,190871,"5'9""",150lbs,Right,91,LW,0,"Aug 3, 2017",NaN,€90M,€270K,€166.5M,408,85,87,62,87,87,448,95,88,89,81,95,453,94,89,96,91,83,357,80,62,81,50,84,356,51,36,87,90,92,93,94,35,30,29,59,9,9,15,15,11,2175,451,5 ★,5★,High,Medium,5 ★,91,85,86,94,36,59,\n273


In [795]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   photoUrl          18979 non-null  object
 1   LongName          18979 non-null  object
 2   playerUrl         18979 non-null  object
 3   Nationality       18979 non-null  object
 4   Positions         18979 non-null  object
 5   Name              18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Team & Contract   18979 non-null  object
 10  ID                18979 non-null  int64 
 11  Height            18979 non-null  object
 12  Weight            18979 non-null  object
 13  foot              18979 non-null  object
 14  BOV               18979 non-null  int64 
 15  BP                18979 non-null  object
 16  Growth            18979 non-null  int64 
 17  Joined      

In [803]:
rename_map = {
    '↓OVA': 'OVA',
    'Team & Contract': 'Team Contract',
    'photoUrl': 'PhotoURL',
    'playerUrl': 'PlayerURL',
    'foot': 'Foot'
}
df = df.rename(columns=rename_map)

In [797]:
# Loop hanya pada kolom object (string) untuk keamanan
obj_cols = df.select_dtypes(include=['object']).columns

for col in obj_cols:
    # Cek apakah kolom memiliki data string sebelum replace
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace('\n', ' ', regex=False)

In [798]:
# Data Type Conversion Functions
# Fungsi Mata Uang (Value, Wage, Release Clause)
def clean_currency(val):
    if pd.isna(val):
        return 0
    
    # Bersihkan simbol & spasi
    val = str(val).replace('€', '').replace(' ', '').upper()
    
    # Logika Multiplier
    if 'M' in val:
        return float(val.replace('M', '')) * 1_000_000
    elif 'K' in val:
        return float(val.replace('K', '')) * 1_000
    else:
        return float(val)

# Fungsi Tinggi Badan (Feet'Inch into cm)
def clean_height(h):
    if pd.isna(h):
        return np.nan
    
    h = str(h).replace('"', '') # Hapus petik dua
    
    try:
        # Jika format feet'inch (cth: 5'7)
        if "'" in h:
            parts = h.split("'")
            feet = float(parts[0])
            # Handle jika inch kosong (cth: 6')
            inches = float(parts[1]) if len(parts) > 1 and parts[1] != '' else 0
            
            total_cm = (feet * 12 + inches) * 2.54
            return round(total_cm) # Output angka bulat (170)
        
        # Jika sudah format cm (contoh: 170cm)
        elif 'cm' in h:
            return float(h.replace('cm', ''))
            
        return float(h)
    except:
        return np.nan

# Fungsi Berat Badan (lbs into kg)
def clean_weight(w):
    if pd.isna(w):
        return np.nan
    
    w = str(w).lower()
    
    if 'lbs' in w:
        val = float(w.replace('lbs', ''))
        return round(val * 0.4536) # Output angka bulat (70)
    elif 'kg' in w:
        return float(w.replace('kg', ''))
        
    return float(w)

In [799]:
# Executing Conversions
# Currency
money_cols = ['Value', 'Wage', 'Release Clause']
for col in money_cols:
    df[col] = df[col].apply(clean_currency)

# Dimensions (Height & Weight)
df['Height in CM'] = df['Height'].apply(clean_height) # Simpan sebagai angka murni
df['Weight in KG'] = df['Weight'].apply(clean_weight) # Simpan sebagai angka murni

# Stars Removal (★)
star_cols = ['W/F', 'SM', 'IR']
for col in star_cols:
    # Hapus bintang, ubah ke int, handle error jika ada string aneh
    df[col] = df[col].astype(str).str.replace('★', '', regex=False).astype(int)

In [800]:
# Date Handling
# Joined Date
df['Joined'] = pd.to_datetime(df['Joined'], errors='coerce')

# Duration (Tahun di Klub)
# Referensi Tahun Data 2021
reference_date = datetime(2021, 1, 1)
df['Years at Club'] = ((reference_date - df['Joined']).dt.days / 365.25).round(1)

# Loan Date End
# Isi NaN dengan 'Unknown' atau NaT (tergantung kebutuhan)
df['Loan Date End'] = df['Loan Date End'].fillna('Not on Loan')

In [801]:
# Drop kolom lama yang sudah tidak dipakai
df = df.drop(columns=['Height', 'Weight']) 

# Set MultiIndex
df = df.set_index(['Nationality', 'Positions']).sort_index()

In [802]:
print("\nDataFrame Multi-Index:")
df.head(5)


DataFrame Multi-Index:


PhotoURL  \
Nationality Positions                                                     
Afghanistan LB         https://cdn.sofifa.com/players/244/175/21_60.png   
            RM ST      https://cdn.sofifa.com/players/243/680/21_60.png   
Albania     CAM        https://cdn.sofifa.com/players/255/448/21_60.png   
            CAM CF     https://cdn.sofifa.com/players/241/577/21_60.png   
            CAM RW LW  https://cdn.sofifa.com/players/247/722/21_60.png   

                              LongName  \
Nationality Positions                    
Afghanistan LB             Hassan Amin   
            RM ST        Omran Haydary   
Albania     CAM            Realdo Fili   
            CAM CF     Laorent Shabani   
            CAM RW LW    Armando Dobra   

                                                               PlayerURL  \
Nationality Positions                                                      
Afghanistan LB         http://sofifa.com/player/244175/hassan-amin/21...   
            RM ST      http://sofifa.com/player/243680/omran-haydary/...   
Albania     CAM        http://sofifa.com/player/255448/realdo-fili/21...   
            CAM CF     http://sofifa.com/player/241577/laorent-shaban...   
            CAM RW LW  http://sofifa.com/player/247722/armando-dobra/...   

                             Name  Age  OVA  POT  \
Nationality Positions                              
Afghanistan LB            H. Amin   28   68   68   
            RM ST      O. Haydary   22   64   71   
Albania     CAM           R. Fili   24   60   64   
            CAM CF     L. Shabani   20   55   67   
            CAM RW LW    A. Dobra   19   57   76   

                                         Team Contract      ID   Foot  BOV  \
Nationality Positions                                                        
Afghanistan LB                 SV Meppen 2018 ~ 2021    244175   Left   68   
            RM ST          Lechia Gdańsk 2020 ~ 2023    243680  Right   65   
Albania     CAM              FC Botoşani 2020 ~ 2024    255448  Right   62   
            CAM CF             IK Sirius 2020 ~ 2021    241577  Right   58   
            CAM RW LW       Ipswich Town 2019 ~ 2023    247722  Right   59   

                        BP  Growth     Joined Loan Date End     Value    Wage  \
Nationality Positions                                                           
Afghanistan LB          LB       0 2018-07-01   Not on Loan 775000.00 1000.00   
            RM ST       RM       7 2020-02-11   Not on Loan 625000.00 2000.00   
Albania     CAM        CAM       4 2020-01-29   Not on Loan 275000.00 2000.00   
            CAM CF      RM      12 2020-03-06   Not on Loan 140000.00  600.00   
            CAM RW LW  CAM      19 2019-02-27   Not on Loan 220000.00 1000.00   

                       Release Clause  Attacking  Crossing  Finishing  \
Nationality Positions                                                   
Afghanistan LB             1100000.00        271        69         49   
            RM ST           969000.00        279        58         66   
Albania     CAM             392000.00        253        48         56   
            CAM CF          203000.00        222        53         42   
            CAM RW LW       523000.00        244        50         51   

                       Heading Accuracy  Short Passing  Volleys  Skill  \
Nationality Positions                                                    
Afghanistan LB                       46             64       43    290   
            RM ST                    35             58       62    272   
Albania     CAM                      26             64       59    283   
            CAM CF                   37             55       35    259   
            CAM RW LW                39             56       48    267   

                       Dribbling  Curve  FK Accuracy  Long Passing  \
Nationality Positions                                                
Afghanistan LB                66     55           53      